In [1]:
# Step1: Importing the required libraries

# linear algebra
import numpy as np
# data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
# for dimensionality reduction
from sklearn.decomposition import PCA

In [2]:
# Step2: Reading the data from train.csv

df_train = pd.read_csv('train.csv')
# Understanding the data
print('Size of training set: {} rows and {} columns'
      .format(*df_train.shape))
# printing few rows and see how the data looks like
df_train.head()

Size of training set: 4209 rows and 378 columns


,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# Step3: Collecting the Y values into an array

# seperating the y from the data as we will use this to learn as 
# the prediction output
y_train = df_train['y'].values


In [4]:
# Step4: Understanding the data types we have

# iterateting through all the columns which has X in the name of the column
cols = [c for c in df_train.columns if 'X' in c]
print('Number of features: {}'.format(len(cols)))

print('Feature types:')
df_train[cols].dtypes.value_counts()

Number of features: 376
Feature types:


int64     368
object      8
Name: count, dtype: int64

In [5]:
# Step5: Counting the data in each of the columns

counts = [[], [], []]
for c in cols:
    typ = df_train[c].dtype
    uniq = len(np.unique(df_train[c]))
    if uniq == 1:
        counts[0].append(c)
    elif uniq == 2 and typ == np.int64:
        counts[1].append(c)
    else:
        counts[2].append(c)

print('Constant features: {} Binary features: {} Categorical features: {}\n'
      .format(*[len(c) for c in counts]))
print('Constant features:', counts[0])
print('Categorical features:', counts[2])

Constant features: 12 Binary features: 356 Categorical features: 8

Constant features: ['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347']
Categorical features: ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']


In [6]:
# Step6: Reading the test.csv data

df_test = pd.read_csv('test.csv')

# remove columns ID and Y from the data as they are not used for learning
usable_columns = list(set(df_train.columns) - set(['ID', 'y']))
y_train = df_train['y'].values
id_test = df_test['ID'].values

x_train = df_train[usable_columns]
x_test = df_test[usable_columns]

In [7]:
# Step7: Checking for null and unique values for test and train sets

def check_missing_values(df):
    if df.isnull().any().any():
        print("There are missing values in the dataframe")
    else:
        print("There are no missing values in the dataframe")
check_missing_values(x_train)
check_missing_values(x_test)

There are no missing values in the dataframe
There are no missing values in the dataframe


In [8]:
# Step8: If for any column(s), the variance is equal to zero, 
# then i've to remove those variable(s).
# Applying label encoder

for column in usable_columns:
    cardinality = len(np.unique(x_train[column]))
    if cardinality == 1:
        x_train.drop(column, axis=1) # Column with only one 
        # value is useless so we drop it
        x_test.drop(column, axis=1)
    if cardinality > 2: # Column is categorical
        mapper = lambda x: sum([ord(digit) for digit in x])
        x_train[column] = x_train[column].apply(mapper)
        x_test[column] = x_test[column].apply(mapper)
x_train.head()

C:\Users\MY PC\AppData\Local\Temp\ipykernel_15264\1743603425.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train[column] = x_train[column].apply(mapper)
C:\Users\MY PC\AppData\Local\Temp\ipykernel_15264\1743603425.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test[column] = x_test[column].apply(mapper)
C:\Users\MY PC\AppData\Local\Temp\ipykernel_15264\1743603425.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

,X238,X104,X19,X255,X333,X379,X94,X148,X68,X156,...,X137,X294,X29,X50,X64,X311,X316,X186,X24,X18
0,0,0,0,0,0,0,0,0,1,1,...,1,0,0,0,0,0,1,0,0,1
1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,1,0,0,1
2,0,0,0,0,0,0,0,1,1,0,...,1,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [9]:
# Step9: Making sure the data is now changed into numericals

print('Feature types:')
x_train[cols].dtypes.value_counts()

Feature types:


int64    376
Name: count, dtype: int64

In [10]:
# Step10: Performing dimensionality reduction
# Linear dimensionality reduction using Singular Value Decomposition of 
# the data to project it to a lower dimensional space.
n_comp = 12
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(x_train)
pca2_results_test = pca.transform(x_test)

In [11]:
# Step11: Training using xgboost

import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(
        pca2_results_train, 
        y_train, test_size=0.2, 
        random_state=4242)

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)
#d_test = xgb.DMatrix(x_test)
d_test = xgb.DMatrix(pca2_results_test)

params = {}
params['objective'] = 'reg:linear'
params['eta'] = 0.02
params['max_depth'] = 4

def xgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

clf = xgb.train(params, d_train, 
                1000, watchlist, early_stopping_rounds=50, 
                feval=xgb_r2_score, maximize=True, verbose_eval=10)

[0]	train-rmse:12.78419	train-r2:0.01323	valid-rmse:11.78154	valid-r2:0.01330
[10]	train-rmse:12.06996	train-r2:0.12040	valid-rmse:10.91823	valid-r2:0.15260
[20]	train-rmse:11.54919	train-r2:0.19467	valid-rmse:10.30543	valid-r2:0.24506
[30]	train-rmse:11.15053	train-r2:0.24931	valid-rmse:9.85657	valid-r2:0.30939
[40]	train-rmse:10.85069	train-r2:0.28914	valid-rmse:9.54286	valid-r2:0.35265
[50]	train-rmse:10.61451	train-r2:0.31975	valid-rmse:9.31132	valid-r2:0.38368


C:\Users\MY PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:726: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)
C:\Users\MY PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(
C:\Users\MY PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [14:34:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[60]	train-rmse:10.42336	train-r2:0.34403	valid-rmse:9.12628	valid-r2:0.40793
[70]	train-rmse:10.18353	train-r2:0.37387	valid-rmse:8.94702	valid-r2:0.43097
[80]	train-rmse:9.98042	train-r2:0.39859	valid-rmse:8.80958	valid-r2:0.44831
[90]	train-rmse:9.80856	train-r2:0.41913	valid-rmse:8.70429	valid-r2:0.46142
[100]	train-rmse:9.67311	train-r2:0.43506	valid-rmse:8.62655	valid-r2:0.47100
[110]	train-rmse:9.55189	train-r2:0.44913	valid-rmse:8.56325	valid-r2:0.47874
[120]	train-rmse:9.43767	train-r2:0.46222	valid-rmse:8.51412	valid-r2:0.48470
[130]	train-rmse:9.34174	train-r2:0.47310	valid-rmse:8.48239	valid-r2:0.48853
[140]	train-rmse:9.25584	train-r2:0.48275	valid-rmse:8.45219	valid-r2:0.49217
[150]	train-rmse:9.15646	train-r2:0.49379	valid-rmse:8.42834	valid-r2:0.49503
[160]	train-rmse:9.07347	train-r2:0.50293	valid-rmse:8.41331	valid-r2:0.49683
[170]	train-rmse:9.00555	train-r2:0.51034	valid-rmse:8.39643	valid-r2:0.49884
[180]	train-rmse:8.93865	train-r2:0.51759	valid-rmse:8.37783	valid

In [12]:
# Step12: Predict your test_df values using xgboost

p_test = clf.predict(d_test)

sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = p_test
sub.to_csv('xgb.csv', index=False)

sub.head()

,ID,y
0,1,82.566696
1,2,94.957817
2,3,81.219086
3,4,76.978302
4,5,113.630463
